In [0]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator

spark = SparkSession.builder.appName('rec').getOrCreate()

In [0]:
data = spark.read.csv('/FileStore/tables/movielens_ratings.csv', inferSchema=True, header=True)

In [0]:
data.show()

+-------+------+------+
|movieId|rating|userId|
+-------+------+------+
|      2|   3.0|     0|
|      3|   1.0|     0|
|      5|   2.0|     0|
|      9|   4.0|     0|
|     11|   1.0|     0|
|     12|   2.0|     0|
|     15|   1.0|     0|
|     17|   1.0|     0|
|     19|   1.0|     0|
|     21|   1.0|     0|
|     23|   1.0|     0|
|     26|   3.0|     0|
|     27|   1.0|     0|
|     28|   1.0|     0|
|     29|   1.0|     0|
|     30|   1.0|     0|
|     31|   1.0|     0|
|     34|   1.0|     0|
|     37|   1.0|     0|
|     41|   2.0|     0|
+-------+------+------+
only showing top 20 rows



In [0]:
data.describe().show()

+-------+------------------+------------------+------------------+
|summary|           movieId|            rating|            userId|
+-------+------------------+------------------+------------------+
|  count|              1501|              1501|              1501|
|   mean| 49.40572951365756|1.7741505662891406|14.383744170552964|
| stddev|28.937034065088994| 1.187276166124803| 8.591040424293272|
|    min|                 0|               1.0|                 0|
|    max|                99|               5.0|                29|
+-------+------------------+------------------+------------------+



In [0]:
train_data, test_data = data.randomSplit([0.9, 0.2])

In [0]:
als = ALS(maxIter=5, regParam=0.01, userCol='userId', itemCol='movieId', ratingCol='rating')

In [0]:
model = als.fit(train_data)

In [0]:
predictions = model.transform(test_data)

In [0]:
predictions.show()

+-------+------+------+-----------+
|movieId|rating|userId| prediction|
+-------+------+------+-----------+
|      3|   1.0|    28|  1.2687213|
|      0|   1.0|    27| -2.3090258|
|      0|   1.0|    22|  1.6339867|
|      0|   1.0|     6|  1.4674019|
|      2|   3.0|     6|  2.1776195|
|      7|   1.0|    16|  3.0647194|
|      4|   2.0|    20|-0.61270195|
|      6|   1.0|    15|  1.0550774|
|      2|   1.0|    17|  1.7614356|
|      5|   1.0|     9|  1.7374809|
|      6|   1.0|     4|   0.724698|
|      2|   4.0|    10| -3.8140364|
|      1|   3.0|    25|-0.32084227|
|      7|   1.0|    25|  1.8517536|
|      7|   1.0|    24| -0.1329473|
|      7|   2.0|    29|   3.560962|
|      0|   1.0|    21| -1.1787107|
|      7|   1.0|    14|  1.0293143|
|      8|   5.0|     2|  0.2513291|
|      1|   1.0|    18| -1.6941946|
+-------+------+------+-----------+
only showing top 20 rows



In [0]:
evaluator = RegressionEvaluator(metricName='rmse', labelCol='rating', predictionCol='prediction')

In [0]:
rmse = evaluator.evaluate(predictions)

In [0]:
rmse

Out[16]: 1.8565518716736942

In [0]:
single_user = test_data.filter(test_data['userId']==11).select(['movieId', 'userId'])

In [0]:
single_user.show()

+-------+------+
|movieId|userId|
+-------+------+
|      9|    11|
|     37|    11|
|     45|    11|
|     67|    11|
|     70|    11|
|     71|    11|
|     77|    11|
|     79|    11|
|     90|    11|
|     97|    11|
+-------+------+



In [0]:
recommendations = model.transform(single_user)

In [0]:
recommendations.orderBy('prediction', ascending=False).show()

+-------+------+----------+
|movieId|userId|prediction|
+-------+------+----------+
|     90|    11|  4.801635|
|     71|    11| 3.9035137|
|      9|    11| 2.4027314|
|     97|    11| 2.0749598|
|     37|    11| 2.0116575|
|     67|    11| 0.8154512|
|     45|    11|0.18179086|
|     77|    11|-1.5770285|
|     70|    11|-1.8469882|
|     79|    11|-2.0798018|
+-------+------+----------+

